## <a id='toc1_1_'></a>[Vision Sensing Application](#toc0_)

In this notebook, we will develop and deploy face detection application to Raspberry Pi4. 

**Table of contents**<a id='toc0_'></a>    
- [Vision Sensing Application](#toc1_1_)    
    - [Prepare Environment](#toc1_1_1_)    
    - [Run Wasm Application](#toc1_1_2_)    
    - [Wasi-Sensor](#toc1_1_3_)    
    - [Deploy Wasm module to Device](#toc1_1_4_)    
    - [Change the application behavior by sending the command](#toc1_1_5_)    
    - [Composite Wasm modules](#toc1_1_6_)    
  - [Thank you for joining the hands-on !](#toc1_2_)    

<!-- vscode-jupyter-toc-config
	numbering=false
	anchor=true
	flat=false
	minLevel=1
	maxLevel=6
	/vscode-jupyter-toc-config -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->

### <a id='toc1_1_1_'></a>[Prepare Environment](#toc0_)

Download the AI model. Today, we use our own TFlite model.


In [ ]:
from urllib.request import urlopen
import os

# Suppress the warning of TFlite
os.environ['PYTHONUNBUFFERED'] = '1'
os.environ['TF_CPP_MIN_LOG_LEVEL']='2'
os.dup2(1, 0)

MODEL_URL=f"http://localhost:4000/face_detection_mobilenet_v2_ssd_lite_fpn_quant.tflite"
MODEL = '/assets/detection.tflite'
with open(file=os.path.join(MODEL), mode="wb") as model_blob:
    download_stream = urlopen(MODEL_URL)
    model_blob.write(download_stream.read())

Prepare the image for inference. Resize the image to 300x300.

In [ ]:
import cv2
import matplotlib.pyplot as plt

IMAGE_URL=f"http://localhost:4000/photo_4-1.jpg"
IMAGE = "./assets/face.png"
with open(file=os.path.join(IMAGE), mode="wb") as image_blob:
    download_stream = urlopen(IMAGE_URL)
    image_blob.write(download_stream.read())

image = cv2.cvtColor(cv2.imread(IMAGE), cv2.COLOR_BGR2RGB)
# Crop [start_row:end_row, start_col:end_col]
image = image[20:320, 330:630] 
image = cv2.resize(image, dsize=(300, 300))


plt.imshow(image)
plt.axis("off")
plt.show()

OK, so we have the Model and the Image.
Let's build the prepared wasm module using VASDK CLI. 

First, please go to ```http://<your-designated-raspberrypi-host>:9000``` and make note of your IP address, which you will copy back here. Also, click there on your designated target port.


In [ ]:
# The IP address you copied from the previous webpage
IP_ADDR = input('Please put your IP address: 10.42.0.1')

Build

In [ ]:
from contextlib import redirect_stdout
import os

if not IP_ADDR:
    raise Exception('Please type your IP address into IP_ADDR in the previous cell')

APP_WORK_SPACE = 'samples/detection-single'
VISION_APP = APP_WORK_SPACE + '/bin/node.wasm'
with redirect_stdout(open(os.devnull, 'w')):
# Since Raspberry Pi 4 is arm64, build wasm and AoT for arm64
    ! vasdk-cli config set webserver.host=$IP_ADDR
    ! cd $APP_WORK_SPACE && vasdk-cli build arm64
print("build done")

Now let's check inside. In this wasm module, there are *Sensor I/F*, *OpenCV I/F*, *Wasi-NN I/F*.

This code is monolithic. Get frames from the camera using Sensor I/F, preprocess image using OpenCV I/F, compute inference using Wasi-NN I/F, and finally report the result using Telemetry.

In [ ]:
!wasm2wat $VISION_APP | grep 'import "'

There are a lot of import functions that we have to load from native libraries in the Device.  So, let's set up the device mock environment.

In [ ]:
from evp_mock.mock import evp
from senscord_mock.mock import senscord
from nn_mock.tflite import wasi_nn
import numpy as np

MODEL="/assets/detection.tflite"
# Specify raspicamv3 camera
stream = "raspicam_image_stream.0"

# Instantiate the mock EVP and SensCord
evp_mock = evp.MockEVP()
sensor_mock = senscord.MockSenscord()
wasi_nn_mock = wasi_nn.WASI_NN(MODEL)

# Back door setting
sensor_mock.set_input(image)
wasi_nn_mock.set_input(np.expand_dims(image.astype(np.float32) / 255.0, axis=0))

# Specify raspicam v3
stream = "raspicam_image_stream.0"
p_param_str = f"{{\"stream\":\"{stream}\",\"model\":\"{MODEL_URL}\"}}"
e = evp.EVPRpc(method="config", params=p_param_str)
evp_mock.injectEvent(e)

# Let's immediately stop for just one frame test
e = evp.EVPShutdown()   
evp_mock.injectEvent(e)  

e = evp.EVPShutdown()   
evp_mock.injectEvent(e)  
print("mock environment is ready")

### <a id='toc1_1_2_'></a>[Run Wasm Application](#toc0_)

Run Wasm Vision and Sensing Application and see what happened

In [ ]:
from node.api import Node

# Start the wasm app 
node = Node()

# Register the native symbols
node.register_natives(evp_mock, sensor_mock)
node.register_nn(wasi_nn_mock)

# Load and instantiate the wasm app
node.load_module(VISION_APP)

# If you want to debug the wasm app, uncomment the following line
# node.start_debugging()
node.main()

This is [detection_single code](samples/detection-single/node/draw_bbox.c). You can modify the contents.

Next, let's run wasm application with debugger and deep dive into the code. In this time, let me introduce the process by reffering the code with debugger.

Now, let's rerun wasm application with the LLDB debugger

In [ ]:
from node.api import Node

p_param_str = f"{{\"stream\":\"{stream}\",\"model\":\"{MODEL_URL}\"}}"
e = evp.EVPRpc(method="config", params=p_param_str)
evp_mock.injectEvent(e)

# Let's immediately stop for just one frame test
# Put the shutdown command in queue
e = evp.EVPShutdown()
evp_mock.injectEvent(e)  
e = evp.EVPShutdown()
evp_mock.injectEvent(e)  

# Start the wasm app 
node = Node()

# Register the native symbols
node.register_natives(evp_mock, sensor_mock)
node.register_nn(wasi_nn_mock)

# Load and instantiate the wasm app
node.load_module(VISION_APP)

# If you want to debug the wasm app, uncomment the following line
node.start_debugging()
node.main()

Select Run->Start Debugging and select "WAMR-Attach", you can start LLDB debugger. 

### <a id='toc1_1_3_'></a>[Wasi-Sensor](#toc0_)

Next, we will deploy exactly same application to Raspberry Pi4.

Thanks to the abstracted Sensor I/F, we can seemlessly move to other device environment.
Imagine being able to deploy your application to any device you want. This is where the **<span style="color:#FFF020">WASI-sensor</span>** interface we are currently developing comes into the conversation.
It is targeting 1D(accerarator, GNSS) to 3D(IMU, Depth) Sensors to be controlled from Wasm.

![wasi-sensor](images/Advanced_Hands-on/wasi-sensor.png)

### <a id='toc1_1_4_'></a>[Deploy Wasm module to Device](#toc0_)

OK, Let's deploy wasm application to Raspberry Pi4. 
Go ahead and undeploy previous wasm application

In [ ]:
! vasdk-cli deploy -e

Then check status

In [ ]:
! vasdk-cli get deployment

Now we deploy detection application

In [ ]:
APP_WORK_SPACE = 'samples/detection-single'
! cd $APP_WORK_SPACE && vasdk-cli -v deploy

Check the status of the application.

In [ ]:
! vasdk-cli get deployment

Specify which stream(sensor) to use. In this case, we use *raspicam_image_stream.0*. 

In [ ]:
stream = "raspicam_image_stream.0"
MODEL_URL=f"http://{IP_ADDR}:4000/face_detection_mobilenet_v2_ssd_lite_fpn_quant.tflite"

# send a message to the wasm app
config = f"{{\"stream\":\"{stream}\",\"model\":\"{MODEL_URL}\"}}"
param_str = f"'{config}'"
! vasdk-cli rpc node "config" $param_str

OK. Now application is running. Let's check telemetry to see face is detected or not.

In [ ]:
! vasdk-cli get telemetry

We can also check the image by using debug viewer. Please open your Browser and access the URL. The window which has your github name will be there.

[http://<<raspi_ip_adder>>:3000](http://192.168.1.62:3000)

### <a id='toc1_1_5_'></a>[Change the application behavior by sending the command](#toc0_)
Let's change the color of bounding box to red.

In [ ]:
! vasdk-cli rpc node "rgb" "FF0000"

Let's go ahead and change the picture quality via VASDK CLI.

Change Saturation to 0. (Default is 255)

In [ ]:
! vasdk-cli rpc node "saturation" "255"

Change Brightness. (Default is 0)

In [ ]:
! vasdk-cli rpc node "brightness" "0"

Change the contrast. (Default is 0)

In [ ]:
! vasdk-cli rpc node "contrast" "0"

OK. application run !! **<span style="color:#FFF020">Congratulation !</span>**

But we know what you feel: Is it an effortless development environment ?  Answer is **<span style="color:#FFF020">NO</span>**.

### <a id='toc1_1_6_'></a>[Composite Wasm module](#toc0_)

We have another type of Vision Sensing Application based on "Nano Process Vision".

[Composite Wasm module for Detection](samples/detection/README.md) is describing the detail.

In [ ]:
from contextlib import redirect_stdout
import os
APP_WORK_SPACE = 'samples/detection'

with redirect_stdout(open(os.devnull, 'w')):
    ! vasdk-cli config set webserver.host=$IP_ADDR
    ! cd $APP_WORK_SPACE && vasdk-cli build arm64
print("build done")

We can check one of the wasm module.


In [ ]:
! wasm2wat $APP_WORK_SPACE/bin/inference_wasi_nn.wasm | grep '(import '

Now obviously dependency on environment is reduced. Reusability and testability is improved.

Each wasm module is loosely connected by topic using the Pub/Sub Messaging pattern.

Let's deploy them to Raspberry Pi4

In [ ]:
! vasdk-cli deploy -e

Check the deployment status

In [ ]:
! vasdk-cli get deployment

Let's deploy now

In [ ]:
APP_WORK_SPACE = 'samples/detection'
! cd $APP_WORK_SPACE && vasdk-cli -v deploy -t 40

Check the deployment status

In [ ]:
! vasdk-cli get deployment

Send RPC command to each wasm module

In [ ]:
# Specify stream(sensor)
stream = "raspicam_image_stream.0"
! vasdk-cli -d rpc senscord_source config $stream

# Specify the location of Model
MODEL_URL=f"http://{IP_ADDR}:4000/face_detection_mobilenet_v2_ssd_lite_fpn_quant.tflite"
! vasdk-cli -d rpc inference_wasi_nn config "http${MODEL_URL}"

Check the inference result. Is it really same as monolithic version ?

## <a id='toc1_2_'></a>[Thank you for joining the hands-on !](#toc0_)